In [1]:
import pandas
import scipy.stats

import hetio.hetnet
import hetio.readwrite
import hetio.stats

from utils import rawgit, open_gz_url, obo_iri

/home/dhimmels/anaconda3/envs/py35/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


## Define the metagraph and instantiate the graph

In [2]:
kind_to_abbev = {
    
    # metanodes
    'compound': 'C',
    'disease': 'D',
    'gene': 'G',
    'anatomy': 'A',
    'symptom': 'S',
    'side effect': 'SE',
    'pathway': 'PW',
    'perturbation': 'PB',
    'biological process': 'BP',
    'cellular component': 'CC',
    'molecular function': 'MF',
    
    # metaedges
    'indication': 'i',
    'binding': 'b',
    'expression': 'e',
    'regulation': 'r',
    'upregulation': 'u',
    'downregulation': 'd',
    'interaction': 'i',
    'evolution': 'e',
    'knockdown downregulation': 'kd',
    'knockdown upregulation': 'ku',
    'overexpression downregulation': 'od',
    'overexpression upregulation': 'ou',
    'participation': 'p',
    'similarity': 's',
    'association': 'a',
    'function': 'f',
    'localization': 'l',
    'presence': 'p',
    'causation': 'c',
}

metaedge_tuples = [
    ('compound', 'disease', 'indication', 'both'),
    ('compound', 'gene', 'binding', 'both'),
    ('compound', 'gene', 'upregulation', 'both'),
    ('compound', 'gene', 'downregulation', 'both'),
    ('compound', 'compound', 'similarity', 'both'),
    ('compound', 'side effect', 'causation', 'both'),
    ('gene', 'anatomy', 'expression', 'both'),
    ('gene', 'anatomy', 'upregulation', 'both'),
    ('gene', 'anatomy', 'downregulation', 'both'),
    ('gene', 'gene', 'interaction', 'both'),
    ('gene', 'gene', 'evolution', 'both'),
    ('gene', 'gene', 'knockdown downregulation', 'forward'),
    ('gene', 'gene', 'knockdown upregulation', 'forward'),
    ('gene', 'gene', 'overexpression downregulation', 'forward'),
    ('gene', 'gene', 'overexpression upregulation', 'forward'),
    ('gene', 'pathway', 'participation', 'both'),
    ('gene', 'perturbation', 'regulation', 'both'),
    ('gene', 'biological process', 'participation', 'both'),
    ('gene', 'cellular component', 'participation', 'both'),
    ('gene', 'molecular function', 'participation', 'both'),
    ('disease', 'disease', 'similarity', 'both'),
    ('disease', 'gene', 'association', 'both'),
    ('disease', 'gene', 'upregulation', 'both'),
    ('disease', 'gene', 'downregulation', 'both'),
    ('disease', 'anatomy', 'localization', 'both'),
    ('disease', 'symptom', 'presence', 'both'),
]
metagraph = hetio.hetnet.MetaGraph.from_edge_tuples(metaedge_tuples, kind_to_abbev)
graph = hetio.hetnet.Graph(metagraph)

## Gene Nodes

In [3]:
commit = '5352b31e04ec136e99d25a0ba63e8867aa71b69f'
url = rawgit('dhimmel', 'entrez-gene', commit, 'data/genes-human.tsv')
gene_df = pandas.read_table(url)
gene_df = gene_df[gene_df.type_of_gene == 'protein-coding']
symbol_to_gene_id = dict(zip(gene_df.Symbol, gene_df.GeneID))
coding_genes = set(gene_df.GeneID)
gene_df.head(2)

,tax_id,GeneID,Symbol,chromosome,map_location,type_of_gene,description
0,9606,1,A1BG,19,19q13.4,protein-coding,alpha-1-B glycoprotein
1,9606,2,A2M,12,12p13.31,protein-coding,alpha-2-macroglobulin


In [4]:
for i, row in gene_df.iterrows():
    if row.type_of_gene != 'protein-coding':
        continue
    data = {'url': 'http://identifiers.org/ncbigene/{}'.format(row.GeneID), 'license': 'CC0 1.0'}
    graph.add_node(kind = 'gene', identifier=row.GeneID, name=row.Symbol, data=data)

## Disease Nodes

In [13]:
commit = '72614ade9f1cc5a5317b8f6836e1e464b31d5587'
url = rawgit('dhimmel', 'disease-ontology', commit, 'data/slim-terms.tsv')
disease_df = pandas.read_table(url)
disease_df.head(2)

,doid,name,source,pathophysiology
0,DOID:2531,Hematologic cancer,DOcancerslim,neoplastic
1,DOID:1319,Brain cancer,DOcancerslim,neoplastic


In [14]:
for i, row in disease_df.iterrows():
    data = {'url': obo_iri(row.doid), 'license': 'CC-BY 3.0'}
    graph.add_node(kind='disease', identifier=row.doid, name=row['name'], data=data)

## Compound Nodes

In [5]:
commit = '3e87872db5fca5ac427ce27464ab945c0ceb4ec6'
url = rawgit('dhimmel', 'drugbank', commit, 'data/drugbank-slim.tsv')
compound_df = pandas.read_table(url)
compound_df.head(2)

,drugbank_id,name,type,groups,atc_codes,categories,inchikey,inchi
0,DB00014,Goserelin,small molecule,approved,L02AE03,NaN,InChIKey=BLCLNMBMMGCOAS-URPVMXJPSA-N,InChI=1S/C59H84N18O14/c1-31(2)22-40(49(82)68-3...
1,DB00035,Desmopressin,small molecule,approved,H01BA02,Antidiuretic Agents|Hemostatics|Renal Agents,InChIKey=NFLWUMRGJYTJIN-NXBWRCJVSA-N,InChI=1S/C46H64N14O12S2/c47-35(62)15-14-29-40(...


In [6]:
for i, row in compound_df.iterrows():
    url = 'http://www.drugbank.ca/drugs/' + row.drugbank_id
    data = {'inchikey': row.inchikey, 'inchi': row.inchi, 'url': url, 'license': 'CC-BY-NC 4.0'}
    graph.add_node(kind='compound', identifier=row.drugbank_id, name=row['name'], data=data)

## Anotomy nodes

In [17]:
commit = '134f23479186abba03ba340fc6dc90e16c781920'
url = rawgit('dhimmel', 'uberon', commit, 'data/hetio-slim.tsv')
uberon_df = pandas.read_table(url)
uberon_df.head(2)

,uberon_id,uberon_name,mesh_id,mesh_name,bto_id
0,UBERON:0000002,uterine cervix,D002584,Cervix Uteri,BTO:0001421
1,UBERON:0000004,nose,D009666,Nose,BTO:0000840


In [18]:
for i, row in uberon_df.iterrows():
    data = {'url': obo_iri(row['uberon_id']), 'license': 'CC-BY 3.0'}
    for xref in 'mesh_id', 'bto_id':
        if pandas.notnull(row[xref]):
            data[xref] = row[xref]
    graph.add_node(kind='anatomy', identifier=row['uberon_id'], name=row['uberon_name'], data=data)

## Symptom Nodes

In [ ]:
commit = 'a7036a37302973b15ab949aab4056d9bc062910e'
url = rawgit('dhimmel', 'mesh', commit, 'data/symptoms.tsv')
symptom_df = pandas.read_table(url)
symptom_df.head(2)

In [ ]:
for i, row in symptom_df.iterrows():
    url = 'http://www.nlm.nih.gov/cgi/mesh/2015/MB_cgi?field=uid&term={}'.format(row.mesh_id)
    data = {'url': url, 'license': 'CC0 1.0'}
    graph.add_node(kind='symptom', identifier=row.mesh_id, name=row.mesh_name, data=data)

## Pathway Nodes and Edges

In [ ]:
commit = '1dc7c744d0d1a8fa17a079f739195e6d3c15117e'
url = rawgit('dhimmel', 'pathways', commit, 'data/pathways.tsv')
pathway_df = pandas.read_table(url)
pathway_df = pathway_df[pathway_df.n_coding_genes > 1]
pathway_df.tail(2)

In [ ]:
for i, row in pathway_df.iterrows():
    pathway_id = row.identifier
    data = {'url': row.url, 'license': row.license}
    graph.add_node(kind='pathway', identifier=pathway_id, name=row['name'], data=data)
    
    for gene in row.coding_genes.split('|'):
        gene = int(gene)
        source_id = 'gene', gene
        target_id = 'pathway', pathway_id
        data = {'unbiased': False, 'url': row.url, 'license': row.license}
        graph.add_edge(source_id, target_id, 'participation', 'both', data)

## Gene Ontology Domains

In [ ]:
commit = '8e5a0a9882fc12780b77f13fd6f86da437a250ba'
url = rawgit('dhimmel', 'gene-ontology', commit, 'annotations/taxid_9606/GO_annotations-9606-inferred-allev.tsv')
go_df = pandas.read_table(url)
go_df.head(2)

In [ ]:
for i, row in go_df.iterrows():
    genes = coding_genes & set(map(int, row.gene_ids.split('|')))
    if 2 > len(genes) or len(genes) > 1000:
        continue
    kind = row['go_domain'].replace('_', ' ')
    data = {'url': obo_iri(row.go_id), 'license': 'CC-BY 4.0'}
    target = graph.add_node(kind=kind, identifier=row['go_id'], name=row['go_name'], data=data)
    target_id = target.get_id()
    for gene in genes:
        source_id = 'gene', gene
        data = {'unbiased': False, 'license': 'CC-BY 4.0'}
        graph.add_edge(source_id, target_id, 'participation', 'both', data)

## Disease-gene associations from compilation

In [12]:
association_df = pandas.read_table('compile/DaG-association.tsv')
association_df.head(2)

,doid_id,doid_name,entrez_gene_id,gene_symbol,sources,license
0,DOID:2531,Hematologic cancer,25,ABL1,DISEASES|DisGeNET,NaN
1,DOID:2531,Hematologic cancer,27,ABL2,DisGeNET,ODbL 1.0


In [15]:
for i, row in association_df.iterrows():
    source_id = 'disease', row.doid_id
    target_id = 'gene', row.entrez_gene_id
    sources = sorted(row.sources.split('|'))
    data = {'sources': sources, 'unbiased': 'GWAS Catalog' in sources}
    if pandas.notnull(license):
        data['license'] = license
    graph.add_edge(source_id, target_id, 'association', 'both', data)

## Disease-gene differential expression

In [ ]:
commit = '4169d25bbc99177d88d0cbd428ae02e886a2d2f9'
url = rawgit('dhimmel', 'adeptus', commit, 'data/gene-sets-slim.tsv')
adeptus_df = pandas.read_table(url)
adeptus_df.head(2)

In [ ]:
for i, row in adeptus_df.iterrows():
    source_id = 'disease', row.slim_id
    target_id = 'gene', row.entrez_gene_id
    kind = row.direction + 'regulation'
    data = {'unbiased': True}
    graph.add_edge(source_id, target_id, kind, 'both', data)

## Chemical similarity

In [ ]:
commit = '3e87872db5fca5ac427ce27464ab945c0ceb4ec6'
url = rawgit('dhimmel', 'drugbank', commit, 'data/similarity-slim.tsv.gz')
chemical_df = pandas.read_table(open_gz_url(url))
chemical_df = chemical_df[chemical_df.similarity >= 0.5]
chemical_df.head(2)

In [ ]:
for i, row in chemical_df.iterrows():
    source_id = 'compound', row.compound0
    target_id = 'compound', row.compound1
    data = {'similarity': row.similarity, 'unbiased': True, 'license': 'CC0 1.0'}
    graph.add_edge(source_id, target_id, 'similarity', 'both', data)

## Symptom edges

In [ ]:
commit = '60d611892bf387b5b23c5f2e2e3bc472cfce85f3'
url = rawgit('dhimmel', 'medline', commit, 'data/disease-symptom-cooccurrence.tsv')
disease_symptom_df = pandas.read_table(url)
disease_symptom_df = disease_symptom_df[disease_symptom_df.p_fisher < 0.005]
disease_symptom_df.head(2)

In [ ]:
for i, row in disease_symptom_df.iterrows():
    source_id = 'disease', row.doid_code
    target_id = 'symptom', row.mesh_id
    data = {'unbiased': False, 'license': 'CC0 1.0'}
    graph.add_edge(source_id, target_id, 'presence', 'both', data)

## Disease-localization edges

In [ ]:
commit = '60d611892bf387b5b23c5f2e2e3bc472cfce85f3'
url = rawgit('dhimmel', 'medline', commit, 'data/disease-uberon-cooccurrence.tsv')
disease_anatomy_df = pandas.read_table(url)
disease_anatomy_df = disease_anatomy_df[disease_anatomy_df.p_fisher < 0.005]
disease_anatomy_df = disease_anatomy_df[disease_anatomy_df.uberon_id.isin(uberon_df['uberon_id'])]

In [ ]:
disease_anatomy_df.head(2)

In [ ]:
for i, row in disease_anatomy_df.iterrows():
    source_id = 'disease', row.doid_code
    target_id = 'anatomy', row.uberon_id
    data = {'unbiased': False, 'license': 'CC0 1.0'}
    graph.add_edge(source_id, target_id, 'localization', 'both', data)

## Disease-disease similarity

In [ ]:
commit = '60d611892bf387b5b23c5f2e2e3bc472cfce85f3'
url = rawgit('dhimmel', 'medline', commit, 'data/disease-disease-cooccurrence.tsv')
disease_similarity_df = pandas.read_table(url)
disease_similarity_df = disease_similarity_df[-disease_similarity_df[['doid_code_0', 'doid_code_1']].apply(frozenset, 1).duplicated()]
disease_similarity_df = disease_similarity_df[disease_similarity_df.p_fisher < 0.005]
disease_similarity_df.head(2)

In [ ]:
for i, row in disease_similarity_df.iterrows():
    source_id = 'disease', row.doid_code_0
    target_id = 'disease', row.doid_code_1
    data = {'unbiased': False, 'license': 'CC0 1.0'}
    graph.add_edge(source_id, target_id, 'similarity', 'both', data)

## Anatomy-gene expression presence

In [19]:
expr_df = pandas.read_table('compile/GeT-expression.tsv.gz', low_memory=False)
expr_df = expr_df[expr_df.uberon_id.isin(uberon_df.uberon_id) & expr_df.entrez_gene_id.isin(coding_genes)]
expr_df.head(2)

,uberon_id,entrez_gene_id,sources,unbiased,license
218662,UBERON:0000002,2,Bgee,1,NaN
218663,UBERON:0000002,9,Bgee,1,NaN


In [ ]:
for i, row in expr_df.iterrows():
    source_id = 'gene', row['entrez_gene_id']
    target_id = 'anatomy', row['uberon_id']
    data = {'unbiased': bool(row['unbiased'])}
    if pandas.notnull(row['license']):
        data['license'] = row['license']
    graph.add_edge(source_id, target_id, 'expression', 'both', data)

## Anatomy-gene differential expression

In [ ]:
commit = '4ceb60f91cd942199ea5453bb14f6b76df4dc158'
url = rawgit('dhimmel', 'bgee', commit, 'data/diffex.tsv.gz')
diffex_df = pandas.read_table(open_gz_url(url))
diffex_df = pandas.melt(diffex_df, id_vars='GeneID', var_name='uberon_id', value_name='direction')
diffex_df = diffex_df.query("direction != 0")
diffex_df = diffex_df[diffex_df.uberon_id.isin(uberon_df.uberon_id) & diffex_df.GeneID.isin(coding_genes)]
diffex_df = diffex_df.replace({'direction': {-1: 'downregulation', 1: 'upregulation'}})
diffex_df.head(2)

In [ ]:
for i, row in diffex_df.iterrows():
    source_id = 'gene', row['GeneID']
    target_id = 'anatomy', row['uberon_id']
    data = {'unbiased': True}
    graph.add_edge(source_id, target_id, row['direction'], 'both', data)

## Compound bindings

In [9]:
binding_df = pandas.read_table('compile/CbG-binding.tsv')
binding_df = binding_df.merge(compound_df[['drugbank_id']])
binding_df = binding_df[binding_df.entrez_gene_id.isin(coding_genes)]
binding_df.head(2)

,drugbank_id,entrez_gene_id,sources,pubmed_ids,actions,affinity_nM,license
0,DB00014,2798,DrugBank (target),20053189,agonist,NaN,NaN
1,DB00014,3973,DrugBank (target),17016423|17139284|20053189,agonist,NaN,NaN


In [11]:
for i, row in binding_df.iterrows():
    source_id = 'compound', row.drugbank_id
    target_id = 'gene', row.entrez_gene_id
    data = {'unbiased': False}
    # singular fields
    for key in 'affinity_nM', 'license':
        value = row[key]
        if pandas.notnull(value):
            data[key] = value
    # compound fields
    for key in 'sources', 'pubmed_ids', 'actions':
        value = row[key]
        if pandas.notnull(value):
            data[key] = value.split('|')
    graph.add_edge(source_id, target_id, 'binding', 'both', data)

## Protein Interactions

In [ ]:
commit = '4012fe7af1699222539844256e3639782ae72695'
url = rawgit('dhimmel', 'ppi', commit, 'data/ppi-hetio-ind.tsv')
ppi_df = pandas.read_table(url)
ppi_df = ppi_df[ppi_df.gene_0.isin(coding_genes) & ppi_df.gene_1.isin(coding_genes)]
ppi_df.head(2)

In [ ]:
for i, row in ppi_df.iterrows():
    source_id = 'gene', row.gene_0
    target_id = 'gene', row.gene_1
    data = {'unbiased': bool(row.unbiased)}
    graph.add_edge(source_id, target_id, 'interaction', 'both', data)

## Evolutionary rate covariation

In [ ]:
commit = '757733f77a89499439c887acb88456e011c5322e'
url = rawgit('dhimmel', 'erc', commit, 'data/erc_mam33-entrez-gt-0.6.tsv.gz')
erc_df = pandas.read_table(open_gz_url(url))
erc_df = erc_df[erc_df.correlation >= 0.75]
erc_df = erc_df[erc_df.source_entrez.isin(coding_genes) & erc_df.target_entrez.isin(coding_genes)]
erc_df.head(2)

In [ ]:
for i, row in erc_df.iterrows():
    source_id = 'gene', row.source_entrez
    target_id = 'gene', row.target_entrez
    data = {'unbiased': True}
    graph.add_edge(source_id, target_id, 'evolution', 'both', data)

## MSigDB -- Perturbations

In [ ]:
commit = 'e8d73491d6b6e8860b73c641dd05d7db1b749467'
url = rawgit('dhimmel', 'msigdb', commit, 'data/msigdb_v5.0.tsv')
msigdb_df = pandas.read_table(url)
msigdb_df.head(2)

In [ ]:
for i, row in msigdb_df[msigdb_df.collection == 'c2.cgp'].iterrows():
    graph.add_node('perturbation', row.gene_set, data={'url': row.url})
    genes = [int(gene) for gene in row.genes.split('|')]
    target_id = 'perturbation', row.gene_set
    for gene in genes:
        source_id = 'gene', gene
        data = {'unbiased': False}
        graph.add_edge(source_id, target_id, 'regulation', 'both', data)

## Indications

In [ ]:
commit = '7c2b17f463babafcf4ec441e720b831340b186fe'
url = rawgit('dhimmel', 'indications', commit, 'data/indications.tsv')
indication_df = pandas.read_table(url)
indication_df = indication_df[indication_df.confidence == 'high']
indication_df.head(2)

In [ ]:
for i, row in indication_df.iterrows():
    source_id = 'disease', row.doid_code
    target_id = 'compound', row.drugbank_id
    data = {'unbiased': False}
    graph.add_edge(source_id, target_id, 'indication', 'both', data)

## LINCS compound-gene expression

In [ ]:
commit = '6eebb4fecc4ae3195fbbc39c8c0f1f4b22b3d79d'
url = rawgit('dhimmel', 'lincs', commit, 'data/consensi/consensi-drugbank.tsv.gz')
l1000_df = pandas.read_table(open_gz_url(url), index_col=0)
l1000_df = l1000_df[l1000_df.index.isin(compound_df.drugbank_id)]
# Bonferroni cutoff
z_cutoff = scipy.stats.norm.ppf(1 - 0.05 / len(l1000_df) / 2)
z_cutoff

In [ ]:
for drugbank_id, row in l1000_df.iterrows():
    upregs = row[row >= z_cutoff]
    downregs = row[row <= -z_cutoff]
    for kind, series in ('upregulation', upregs), ('downregulation', downregs):
        for gene, zscore in series.items():
            source_id = 'compound', drugbank_id
            target_id = 'gene', int(gene)
            data = {'z-score': zscore, 'unbiased': True}
            graph.add_edge(source_id, target_id, kind, 'both', data)

## LINCS knockdowns

In [ ]:
url = rawgit('dhimmel', 'lincs', commit, 'data/consensi/consensi-knockdown.tsv.gz')
l1000_kd_df = pandas.read_table(open_gz_url(url), index_col=0)
l1000_kd_df = l1000_kd_df.groupby(symbol_to_gene_id).mean()

In [ ]:
for knockdown, row in l1000_kd_df.iterrows():
    upregs = row[row >= z_cutoff]
    downregs = row[row <= -z_cutoff]
    for kind, series in ('knockdown upregulation', upregs), ('knockdown downregulation', downregs):
        for gene, zscore in series.items():
            source_id = 'gene', int(knockdown)
            target_id = 'gene', int(gene)
            if source_id == target_id:
                continue
            data = {'z-score': zscore, 'unbiased': True}
            graph.add_edge(source_id, target_id, kind, 'forward', data)

## LINCS overexpressions

In [ ]:
url = rawgit('dhimmel', 'lincs', commit, 'data/consensi/consensi-overexpression.tsv.gz')
l1000_oe_df = pandas.read_table(open_gz_url(url), index_col=0)
l1000_oe_df = l1000_oe_df.groupby(symbol_to_gene_id).mean()

In [ ]:
for overexpression, row in l1000_oe_df.iterrows():
    upregs = row[row >= z_cutoff]
    downregs = row[row <= -z_cutoff]
    for kind, series in ('overexpression upregulation', upregs), ('overexpression downregulation', downregs):
        for gene, zscore in series.items():
            source_id = 'gene', int(overexpression)
            target_id = 'gene', int(gene)
            if source_id == target_id:
                continue
            data = {'z-score': zscore, 'unbiased': True}
            graph.add_edge(source_id, target_id, kind, 'forward', data)

## Side Effects - SIDER

In [ ]:
commit = '2acca0b065e736bc99702906024efd4718e502ee'
url = rawgit('dhimmel', 'SIDER4', commit, 'data/side-effect-terms.tsv')
side_effect_df = pandas.read_table(url)
side_effect_df.head(2)

In [ ]:
for i, row in side_effect_df.iterrows():
    umls_id = row['umls_cui_from_meddra']
    data = {'url': 'http://sideeffects.embl.de/se/{}/'.format(umls_id), 'license': 'CC-BY-NC-SA 4.0'}
    graph.add_node(kind='side effect', identifier=umls_id, name=row['side_effect_name'])

In [ ]:
url = rawgit('dhimmel', 'SIDER4', commit, 'data/side-effects.tsv')
sider_df = pandas.read_table(url)
sider_df = sider_df[sider_df.drugbank_id.isin(compound_df.drugbank_id)]
sider_df.head(2)

In [ ]:
for i, row in sider_df.iterrows():
    source_id = 'compound', row.drugbank_id
    target_id = 'side effect', row.umls_cui_from_meddra
    data = {'unbiased': False, 'license': 'CC-BY-NC-SA 4.0'}
    graph.add_edge(source_id, target_id, 'causation', 'both', data)

## Summary

In [ ]:
metanode_to_nodes = graph.get_metanode_to_nodes()
for metanode, nodes in metanode_to_nodes.items():
    print(metanode, len(nodes))

In [ ]:
metaedge_to_edges = graph.get_metaedge_to_edges(exclude_inverts=True)
for metaedge, edges in metaedge_to_edges.items():
    print(metaedge, len(edges))

## Save graph

In [ ]:
# Write nodes to a table
path = 'data/nodes.tsv'
hetio.readwrite.write_nodetable(graph, path)

# Write edges to a table
path = 'data/edges.sif.gz'
hetio.readwrite.write_sif(graph, path)

In [ ]:
# Write a subset of edges to a table
path = 'data/edges-10.sif'
hetio.readwrite.write_sif(graph, path, max_edges=10)

path = 'data/edges-5k.sif.gz'
hetio.readwrite.write_sif(graph, path, max_edges=5000)

In [ ]:
# Write graph as json
path = 'data/hetnet.json.gz'
hetio.readwrite.write_json(graph, path)

In [ ]:
! sha256sum data/hetnet.json.gz

# Network visualizations and stats

In [ ]:
# import imp
# imp.reload(hetio.hetnet)
# imp.reload(hetio.stats)

In [ ]:
# Create and save degree distribution vizualizations
hetio.stats.plot_degrees(graph, 'viz/degrees.pdf')

In [ ]:
# Summary of metanodes and cooresponding nodes
metanode_df = hetio.stats.get_metanode_df(graph)
metanode_df.to_csv('data/summary/metanodes.tsv', sep='\t', index=False)
metanode_df

In [ ]:
metanode_df.sum()

In [ ]:
# Summary of metaedges and cooresponding edges
metaedge_df = hetio.stats.get_metaedge_df(graph)

# Calculate number of unbiased edges
rows = list()
for metaedge, edges in graph.get_metaedge_to_edges().items():
    unbiased = sum(edge.data['unbiased'] for edge in edges)
    rows.append({'metaedge': str(metaedge), 'unbiased': unbiased})

metaedge_df = metaedge_df.merge(pandas.DataFrame(rows))
metaedge_df.to_csv('data/summary/metaedges.tsv', sep='\t', index=False)
metaedge_df.query('inverted == 0').reset_index(drop=True)

In [ ]:
metaedge_df.sum()

## Barplots of metaedge and metanode counts

In [ ]:
import seaborn
%matplotlib inline

In [ ]:
ax = seaborn.barplot(x='metanode', y='nodes', data=metanode_df.sort('nodes'))
for tick in ax.get_xticklabels():
    tick.set_rotation(90)
ax.set_xlabel(''); ax.set_ylabel('nodes');

In [ ]:
ax = seaborn.barplot(x='metaedge', y='edges', data=metaedge_df.sort('edges').query('inverted == 0'))
for tick in ax.get_xticklabels():
    tick.set_rotation(90)
ax.set_xlabel(''); ax.set_ylabel('edges');